# forensicface--A tool for forensic face examination

> An integrated tool to compare faces using state-of-the-art face recognition models and compute Likelihood Ratios 

In [1]:
# | default_exp app

In [1]:
# | export
from nbdev.showdoc import *
from fastcore.utils import *
import onnxruntime
import cv2
import numpy as np
import os.path as osp
from insightface.app import FaceAnalysis
from insightface.utils import face_align

In [2]:
# | export
class ForensicFace:
    "A (forensic) face comparison tool"
    
    def __init__(
        self, model: str = "sepaelv2", det_size: int = 320, use_gpu: bool = True
    ):

        self.det_size = (det_size, det_size)
        self.detectmodel = FaceAnalysis(
            name=model,
            allowed_modules=["detection"],
            providers=["CUDAExecutionProvider"]
            if use_gpu
            else ["CPUExecutionProvider"],
        )
        self.detectmodel.prepare(ctx_id=0 if use_gpu else -1, det_size=self.det_size)
        self.ort_session = onnxruntime.InferenceSession(
            osp.join(
                osp.expanduser("~/.insightface/models"),
                model,
                "adaface",
                "adaface_ir101web12m.onnx",
            ),
            providers=["CUDAExecutionProvider"]
            if use_gpu
            else ["CPUExecutionProvider"],
        )

    def _to_input(self, aligned_bgr_img):
        _aligned_bgr_img = aligned_bgr_img.astype(np.float32)
        _aligned_bgr_img = ((_aligned_bgr_img / 255.0) - 0.5) / 0.5
        return _aligned_bgr_img.transpose(2, 0, 1).reshape(1, 3, 112, 112)

    def get_most_central_face(self, img, faces):
        """
        faces is a insightface object with keypoints and bounding_box

        return: keypoints of the most central face
        """
        assert faces is not None
        img_center = np.array([img.shape[0] // 2, img.shape[1] // 2])
        dist = []

        # Compute centers of faces and distances from certer of image
        for idx, face in enumerate(faces):
            box = face.bbox.astype("int").flatten()
            face_center = np.array([(box[0] + box[2]) // 2, (box[1] + box[3]) // 2])
            dist.append(np.linalg.norm(img_center - face_center))

        # Get index of the face closest to the center of image
        return faces[dist.index(min(dist))].kps

    def process_image(self, imgpath: str):  # Path to image to be processed
        """
        Process image and returns list of dicts with:

        - keypoints: 5 facial points (left eye, right eye, nose tip, left mouth corner and right mouth corner)

        - ipd: interpupillary distance

        - normalized_embedding

        - embedding_norm

        - aligned_face: face after alignment using the keypoints as references for affine transform
        """
        bgr_img = cv2.imread(imgpath)
        faces = self.detectmodel.get(bgr_img)
        if len(faces) == 0:
            return {}
        kps = self.get_most_central_face(bgr_img, faces)
        bgr_aligned_face = face_align.norm_crop(bgr_img, kps)
        ipd = np.linalg.norm(kps[0] - kps[1])
        ort_inputs = {
            self.ort_session.get_inputs()[0].name: self._to_input(bgr_aligned_face)
        }
        normalized_embedding, norm = self.ort_session.run(None, ort_inputs)
        return {
            "keypoints": kps,
            "ipd": ipd,
            "embedding": normalized_embedding.flatten() * norm.flatten()[0],
            "norm": norm.flatten()[0],
            "aligned_face": cv2.cvtColor(bgr_aligned_face, cv2.COLOR_BGR2RGB),
        }

In [11]:
ff = ForensicFace(use_gpu=True)
ff

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'do_copy_in_default_stream': '1', 'arena_extend_strategy': 'kNextPowerOfTwo', 'gpu_external_empty_cache': '0', 'gpu_external_free': '0', 'cudnn_conv_use_max_workspace': '0', 'gpu_mem_limit': '18446744073709551615', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'gpu_external_alloc': '0', 'device_id': '0'}}
find model: /home/rafael/.insightface/models/sepaelv2/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
set det-size: (320, 320)


In [14]:
results = ff.process_image("obama.png")
results.keys(), results['keypoints']

(dict_keys(['keypoints', 'ipd', 'embedding', 'norm', 'aligned_face']),
 array([[103.600136, 139.8824  ],
        [174.26506 , 137.33737 ],
        [140.28088 , 187.14763 ],
        [109.09409 , 219.34001 ],
        [173.40773 , 217.09573 ]], dtype=float32))

## Comparação entre duas imagens

In [3]:
# | export
@patch
def compare(self:ForensicFace, img1path: str, img2path: str):
    img1data = self.process_image(img1path)
    assert len(img1data) == 5
    img2data = self.process_image(img2path)
    assert len(img2data) == 5
    return np.dot(img1data["embedding"], img2data["embedding"]) / (
        img1data["norm"] * img2data["norm"]
    )

In [15]:
ff.compare("obama.png","obama2.png")

0.8556977

## Agregação de embeddings

In [5]:
# | export
@patch
def aggregate_embeddings(self:ForensicFace, embeddings, weights=None):
    if weights is None:
        weights = np.ones(embeddings.shape[0], dtype="int")
    assert embeddings.shape[0] == weights.shape[0]
    return np.average(embeddings, axis=0, weights=weights)


In [6]:
# | export
@patch
def aggregate_from_images(self:ForensicFace, list_of_image_paths):
    embeddings = []
    weights = []
    for imgpath in list_of_image_paths:
        d = self.process_image(imgpath)
        embeddings.append(d["embedding"])
    return self.aggregate_embeddings(np.array(embeddings))

In [17]:
aggregated = ff.aggregate_from_images(["obama.png","obama2.png"])
aggregated.shape

(512,)